In [1]:
import numpy as np
import wandb
import pandas as pd
from tqdm import tqdm

In [2]:
api = wandb.Api(timeout=600)

In [3]:
runs = api.runs("inchangbaek4907/scenario")
len(runs)

222

In [4]:
runs = list(runs)
runs[:5]

[<Run inchangbaek4907/scenario/re2wd1xx (finished)>,
 <Run inchangbaek4907/scenario/h5m442q9 (finished)>,
 <Run inchangbaek4907/scenario/ak7yr998 (finished)>,
 <Run inchangbaek4907/scenario/mwu8l4yk (finished)>,
 <Run inchangbaek4907/scenario/7ek9ue01 (finished)>]

In [5]:

class RunInfo:
    config: dict
    name: str
    run_object: wandb.apis.public.Run
    
    def __init__(self, config, name, run_object):
        self.config = config
        self.name = name
        self.run_object = run_object

    def __str__(self):
        showing_config = dict()
        showing_keys = ['gpt_model']
        
        for key in showing_keys:
            if key in self.config:
                showing_config[key] = self.config[key]
        
        return f"RunInfo(\n\tname={self.name},\n\tconfig={showing_config}\n)"

In [6]:
run_infos = list()

for run in tqdm(runs):
    if run.state == 'finished' and run.summary['Evaluation/llm_iteration'] < 6:
        run_infos.append(RunInfo(run.config, run.name, run))

len(run_infos)

100%|██████████| 222/222 [00:00<00:00, 67807.71it/s]


0

In [7]:
configs = list()
for run_info in tqdm(run_infos[:]):
    configs.append(run_info.config)
    
df = pd.DataFrame(configs)
df

0it [00:00, ?it/s]


""


In [8]:
commands = list()

def get_config(exp_group, target_character):
    return df[(df['exp_group'] == exp_group) & (df['target_character'] == target_character)].iloc[0]

for index, row in df.iterrows():
    
    config = get_config(row['exp_group'], row['target_character'])
    
    seed = row['seed']
    exp_group = row['exp_group']
    target_character = row['target_character']
    
    total_iterations = config['total_iterations']
    gpt_model = config['gpt_model']
    exp_name = config['exp_name']
    n_self_alignment = config['n_self_alignment']
    evaluator = config['evaluator']
    pe = config['pe']
    
    # if nan
    if pd.isna(n_self_alignment):
        n_self_alignment = 0
    n_self_alignment = int(n_self_alignment)
    

    command = f"bash sbatch_gpu.sh experiment.py n_envs=600 wandb_project=scenario task=scenario overwrite=True pe={pe} exp_name={exp_name} gpt_model={gpt_model} total_iterations={total_iterations} seed={seed} evaluator={evaluator} target_character={target_character} n_self_alignment={n_self_alignment}"
    commands.append(command)

commands

[]

In [9]:
commands_block = " \n".join(commands)
from IPython.display import display, Markdown
display(Markdown("```bash\n" + commands_block + ""))

```bash
